In [ ]:
pip install numpy-stl

In [ ]:
# STL bounding box
# pip install numpy-stl
import numpy as np
from stl import mesh
import gzip
import os

def is_compressed(file_path):
    return file_path.endswith('.gz')

def decompress_stl_gz(gz_file_path, decompressed_file_path):
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(decompressed_file_path, 'wb') as f_out:
            f_out.write(f_in.read())

def get_bounding_box(stl_file_path):
    # Load the STL file
    your_mesh = mesh.Mesh.from_file(stl_file_path)

    # Get the minimum and maximum coordinates along each axis
    min_x, max_x = np.min(your_mesh.x), np.max(your_mesh.x)
    min_y, max_y = np.min(your_mesh.y), np.max(your_mesh.y)
    min_z, max_z = np.min(your_mesh.z), np.max(your_mesh.z)

    # Define the bounding box
    bounding_box = {
        'hullXmin': min_x, 'hullXmax': max_x,
        'hullYmin': min_y, 'hullYmax': 0,
        'hullZmin': min_z, 'hullZmax': max_z
    }

    return bounding_box

def write_bounding_box_to_file(bounding_box, file_path):
    with open(file_path, 'w') as file:
        for key, value in bounding_box.items():
            file.write(f"{key}  {value:.4f};\n")

# Example usage
file_path = 'hullDTC.stl'
decompressed_file_path = 'decompressed_file.stl'
output_file_path = 'bounding_box.txt'

if is_compressed(file_path):
    # Decompress the STL file if it is compressed
    decompress_stl_gz(file_path, decompressed_file_path)
    stl_file_path = decompressed_file_path
else:
    stl_file_path = file_path

# Get the bounding box of the STL file
bounding_box = get_bounding_box(stl_file_path)

# Write the bounding box to a file
write_bounding_box_to_file(bounding_box, output_file_path)

# Remove the decompressed file if it was created
if is_compressed(file_path):
    os.remove(decompressed_file_path)

print(f"Bounding box data written to {output_file_path}")


In [ ]:
import bpy
import bmesh

def load_stl(filepath):
    bpy.ops.import_mesh.stl(filepath=filepath)

def cut_geometry(draft):
    # Ensure we are in object mode
    if bpy.context.object.mode != 'OBJECT':
        bpy.ops.object.mode_set(mode='OBJECT')

    obj = bpy.context.active_object

    # Create a bmesh from the object
    bm = bmesh.new()
    bm.from_mesh(obj.data)

    # Cut the geometry using a Z-plane at Z=0
    plane_co = (0, 0, draft)
    plane_no = (0, 0, 1)
    bmesh.ops.bisect_plane(bm, geom=bm.verts[:] + bm.edges[:] + bm.faces[:], plane_co=plane_co, plane_no=plane_no, clear_outer=True)

    # Update the mesh
    bm.to_mesh(obj.data)
    bm.free()

def remove_top_part():
    obj = bpy.context.active_object

    # Go into edit mode
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.mesh.select_all(action='DESELECT')

    # Select all vertices above Z=0
    bpy.ops.mesh.select_by_location(location=(0, 0, 0), selection_type='ABOVE_THRESHOLD')

    # Delete the selected vertices
    bpy.ops.mesh.delete(type='VERT')
    bpy.ops.object.mode_set(mode='OBJECT')

# Replace with the path to your STL file
stl_path = "hullDTC.stl"
draft = 0.244

# Load the STL file
load_stl(stl_path)

# Cut the geometry
cut_geometry(draft)

# Remove the top part
remove_top_part()

# Export the modified STL
output_path = "hull_clipped_blender.stl"
bpy.ops.export_mesh.stl(filepath=output_path)


In [3]:
import bpy
import bmesh

# Adjust this path to your STL file
stl_path = 'hullDTC.stl'
output_path = 'hull_clipped_blender.stl'
z_plane_height = 0.244  # Adjust this value as needed

# Open a new blank file (without default cube and camera)
bpy.ops.wm.read_factory_settings(use_empty=True)

# Import the STL file
bpy.ops.import_mesh.stl(filepath=stl_path)

# Assuming the imported mesh is the only object in the scene
obj = bpy.context.selected_objects[0]
bpy.context.view_layer.objects.active = obj

# Enter edit mode
bpy.ops.object.mode_set(mode='EDIT')

# Create a bmesh object from the mesh
bm = bmesh.from_edit_mesh(obj.data)

# Bisect the mesh using the z-plane
bpy.ops.mesh.bisect(
    plane_co=(0, 0, z_plane_height),
    plane_no=(0, 0, 1),
    use_fill=False,
    clear_outer=True
)

# Update the mesh and exit edit mode
bmesh.update_edit_mesh(obj.data)
bpy.ops.object.mode_set(mode='OBJECT')

# Save the modified mesh as an STL file
bpy.ops.export_mesh.stl(filepath=output_path)

print(f'Modified STL saved to {output_path}')


Import finished in 1.5625 sec.
Modified STL saved to hull_clipped_blender.stl
